In [12]:
import pandas as pd
import numpy as np
import recmetrics
import matplotlib.pyplot as plt
from models import metrics

In [13]:
# map@K, recall@K

K = 10 
test = pd.read_csv('test.csv', encoding='utf-8')

In [14]:
dff = test.copy()

df = pd.DataFrame(columns=['user_id', 'actual','recommendations'])
test_list_lists = []
recs_list_lists = []
users_list = []

for userid in dff["user_id"].sort_values():
    
    test_list = dff[dff["user_id"]==userid].actual.values[0].replace('[', '').replace(']', '')
    recs_list = dff[dff["user_id"]==userid].recommendations.values[0].replace('[', '').replace(']', '')
    test_list = [int(x.strip()) for x in test_list.split(',') if x.strip()]
    recs_list = [int(x.strip()) for x in recs_list.split(',') if x.strip()]

    df=pd.concat([pd.DataFrame([
            {   
                'user_id': userid, 
                'actual': test_list, 
                'recommendations': recs_list
             }]),
            df],ignore_index=True)
    
    test_list_lists.append(test_list)
    recs_list_lists.append(recs_list)
    users_list.append(userid)

# type(df[df["user_id"]==userid].actual.values[0])

In [15]:
main_df = df.sort_values(by="user_id").reset_index(drop=True)
# main_df

### recall@K

In [33]:
%time
#my code

def my_recommender_recall(predicted, actual):
    
    def recall_func(actual,predicted): # tp/(tp+fn)
        actual = set(actual)
        predicted = set(predicted)
        tp_set = actual & predicted
        fn_set = actual - tp_set
        tp = len(tp_set)
        fn = len(fn_set)
        return tp/(tp+fn)

    return list(map(recall_func, actual, predicted))

recall_list = my_recommender_recall(predicted=recs_list_lists, actual=test_list_lists)
my_recall = np.mean(recall_list)
my_recall

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.05 µs


0.8965933357563518

In [32]:
%time
#recmetrics code

def recommender_recall(predicted, actual):
    
    def calc_recall(predicted, actual):
        reca = [value for value in predicted if value in actual]
        reca = float(len(reca)) / float(len(actual))
        return reca
    
    return list(map(calc_recall, predicted, actual))

rec_recall_list = recommender_recall(predicted=recs_list_lists, actual=test_list_lists)
rec_recall = np.mean(rec_recall_list)
rec_recall

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 5.25 µs


0.8964706865331302

### AP@K

In [6]:

ap_list = []

for i in range(len(users_list)):
    ap_list.append(metrics._apk(test_list_lists[i],recs_list_lists[i]))

my_ap = sum(ap_list)/len(ap_list)
main_df['ap'] = ap_list
my_ap

0.8905710370328134

In [7]:
np.mean(ap_list)

0.8905710370328139

In [8]:
main_df

,user_id,actual,recommendations,recall,ap
0,0,[95],"[9, 38, 32, 160, 0, 61, 179, 180, 95, 667]",1.0,0.111111
1,1,"[8, 25, 271]","[8, 25, 271, 0, 948, 412, 947, 1146, 826, 775]",1.0,1.000000
2,2,"[15, 136, 195]","[9, 195, 24, 136, 79, 282, 281, 15, 283, 372]",1.0,0.458333
3,3,"[21, 109, 313]","[109, 0, 313, 51, 21, 53, 356, 142, 445, 978]",1.0,0.755556
4,4,[0],"[11, 132, 118, 61, 97, 999999, 388, 121, 147, ...",0.0,0.000000
...,...,...,...,...,...
4887,4887,[1259],"[1259, 470, 1361, 1362, 893, 1048, 557, 22, 26...",1.0,1.000000
4888,4888,[1338],"[999999, 587, 216, 978, 72, 317, 332, 820, 561...",0.0,0.000000
4889,4889,[1368],"[1368, 759, 1024, 228, 897, 499, 518, 274, 331...",1.0,1.000000
4890,4890,[1448],"[1448, 889, 43, 164, 297, 800, 1127, 1002, 133...",1.0,1.000000


### map@K

## recmetrics

In [16]:
mark = metrics.mark(actual=test_list_lists,predicted=recs_list_lists,k=K)
mapk = metrics.mapk(actual=test_list_lists,predicted=recs_list_lists,k=K)
rrec = metrics.recommender_recall(actual=test_list_lists,predicted=recs_list_lists)
rprc = metrics.recommender_precision(actual=test_list_lists,predicted=recs_list_lists)

print(f"mark @{K}      = {mark}")
print(f"mapk @{K}      = {mapk}")
print(f"rec recall    = {rrec}")
print(f"rec precision = {rprc}")

mark @10      = 0.8227200303443132
mapk @10      = 0.8905710370328139
rec recall    = 0.8964706865332176
rec precision = 0.38808258381030253


# Overall

In [35]:
# recall

print("*"*50)
print(f"My Metrics:")
print(f"Mean Recall                     = {my_recall:.4f}")
# print(f"Mean Average precision          = {my_ap}")

print("*"*50)
print(f"Recmetrics:")
print(f"Recommender Recall (the same)   = {rrec:.4f}")
print(" "*50)
print(f"MAR@K (mean average recall)     = {mark:.4f}")
print(f"Recommender Precision           = {rprc:.4f}")
print(f"MAP@K (mean average precision)  = {mapk:.4f}")


**************************************************
My Metrics:
Mean Recall                     = 0.8966
**************************************************
Recmetrics:
Recommender Recall (the same)   = 0.8965
                                                  
MAR@K (mean average recall)     = 0.8227
Recommender Precision           = 0.3881
MAP@K (mean average precision)  = 0.8906


# Main Table

In [11]:
pd.set_option('display.max_colwidth', None)
main_df.head(30)

,user_id,actual,recommendations,recall,ap
0,0,[95],"[9, 38, 32, 160, 0, 61, 179, 180, 95, 667]",1.000000,0.111111
1,1,"[8, 25, 271]","[8, 25, 271, 0, 948, 412, 947, 1146, 826, 775]",1.000000,1.000000
2,2,"[15, 136, 195]","[9, 195, 24, 136, 79, 282, 281, 15, 283, 372]",1.000000,0.458333
3,3,"[21, 109, 313]","[109, 0, 313, 51, 21, 53, 356, 142, 445, 978]",1.000000,0.755556
4,4,[0],"[11, 132, 118, 61, 97, 999999, 388, 121, 147, 193]",0.000000,0.000000
5,5,[0],"[0, 142, 445, 313, 542, 913, 180, 482, 556, 388]",1.000000,1.000000
6,6,[0],"[0, 142, 445, 313, 542, 913, 180, 482, 556, 388]",1.000000,1.000000
7,7,[0],"[0, 142, 445, 313, 542, 913, 180, 482, 556, 388]",1.000000,1.000000
8,8,"[0, 188, 352]","[188, 538, 612, 157, 227, 518, 126, 566, 73, 533]",0.333333,1.000000
9,9,"[38, 60]","[38, 60, 86, 64, 9, 0, 146, 145, 833, 402]",1.000000,1.000000
